In [1]:
#Library
import pandas as pd 
from pmaw import PushshiftAPI
import datetime as dt
import requests
import json

In [2]:
api = PushshiftAPI()

In [2]:
#convert date to epoch time 

before = int(dt.datetime(2020,12,22,0,0).timestamp()) 
start_time =  int(dt.datetime(2021,3,6,0,0).timestamp())
fantasy_bball = "fantasybball"
#wsb = 
limit = 10_000

fantasy_bball = api.search_submissions(subreddit=fantasy_bball, 
                                     #limit=limit,
                                     before=start_time,
                                     after=before,
                                     filter=['title', 'selftext', 'id' , 'subreddit', 'url', 'removed_by_category']
                                     ) #'author'

#print(f'Retrieved {len(submissions)} comments from Pushshift')
fantasy_bball_df = pd.DataFrame(fantasy_bball)

NameError: name 'dt' is not defined

In [1]:
fantasy_bball_df.tail()

NameError: name 'fantasy_bball_df' is not defined

In [9]:
post_id = list(submissions_df.id)
#post_id

In [10]:
#get all the comments ids for our submissions
comment_ids = api.search_submission_comment_ids(ids=post_id)

comment_id_list = [c_id for c_id in comment_ids]
len(comment_id_list)

Total:: Success Rate: 98.73% - Requests: 79 - Batches: 8 - Items Remaining: 0


2940

In [11]:
#Take the comments ids and find the body. 
comments = api.search_comments(ids=comment_id_list,
                               limit=1000,
                               filter=['author', 'body', 'subreddit', 'id', 'link_id'])
#comment_list = [comment for comment in comments]
df_comments = pd.DataFrame(comments)
df_comments 

Total:: Success Rate: 100.00% - Requests: 3 - Batches: 1 - Items Remaining: 0


,author,body,id,link_id,subreddit
0,megatronVI,"Yep , seems like a decoy. Schroeder, THT, Lebr...",ghczued,t3_km6nzb,fantasybball
1,SnaxFoods,"could’ve had Harden, AD gives me pain",ghczvl4,t3_km6nzb,fantasybball
2,gkaourisss,"It was either AD or Lillard for me, I should h...",ghczzfo,t3_km6nzb,fantasybball
3,d34th_boltz,Something is absolutely wrong with AD right no...,ghczzye,t3_km6nzb,fantasybball
4,jackmakesblackjacks,Dont worry. AD will get his. At 5 hes an absol...,ghd00h1,t3_km6nzb,fantasybball
...,...,...,...,...,...
2935,NectarCollector808,I jus picked him off waivers in a 8 team cat h...,gikkrrc,t3_kmcvkz,fantasybball
2936,niqoflous,Looks like he just had another solid outing ye...,gikzw9r,t3_kmb3ik,fantasybball
2937,rhyder78,I picked him up in my 16x16 league.,gil8xs7,t3_kmb3ik,fantasybball
2938,[deleted],[removed],gl0r692,t3_kmauim,fantasybball


In [14]:
#grouping comments for each posts and concatenating them together
#One row per post, with that row con taining all the comments on that post. 

grouped_comments = df_comments.groupby('link_id').agg({'body': '\n'.join})
grouped_comments.body[1]

'I wouldn’t give harden away for anything at this moment in time.\nThere’s a thread for trades\nI don’t really know how to get there I’m new to this app\nSearch trade thread on this sub\nCan you point me to it too? Last I checked it was a bunch of empty comments from real accounts\nAnd OP, if ast are worth more than boards then I’d keep Harden and Trae\nRebounds are 1 assists are 2 but steals and blocks are 4 each\nFeel you but Trae and Harden hit too many 3’s too ignore imo\nYeah I agree with u there but I really feel like Damian lillard might start going off'

In [29]:

#fantasy_bball_df.id = "t3_" + submissions_df.id
#fantasy_bball_df.id[6001]

nan

In [17]:
sum(pd.isnull(fantasy_bball_df['id']))

#Conclusion there are a lot of NaN value for id (5978). 


5978

In [32]:
import re 
import numpy as np 
import pandas as pd 

#Gensim 
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization 
import spacy 

#plotting tools 
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt 


import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from tqdm import tqdm_notebook as tqdm #allows us to create progress bars to track how long it,s taking to process.
from pprint import pprint #helps for formating

In [34]:
# Prepare stopwords 

# NLTK stop words
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')

stop_words = stopwords.words('english')
#if i want to add other stop words based on the topic.
#stopwords.extend(['from', 'reddit'])

In [56]:
# Convert to list 

data_fantasybball_title = fantasy_bball_df.selftext.tolist()
# Remove distracting single quotes
#data_fantasybball_title = [re.sub("\'", "", sent) for sent in data_fantasybball_title]

print(data_fantasybball_title[:1])

['How long will it take for Espn to update the positions eligibility for Lonzo and Bledsoe? They’re both listed as point guards and they start alongside each other']


In [57]:
#Tokenize words and clean-up text 

def sent_to_words(sentences):
    for sentence in sentences:
         # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence))) 

data_bball_words = list(sent_to_words(data_fantasybball_title))

print(data_bball_words[:1])

[['how', 'long', 'will', 'it', 'take', 'for', 'espn', 'to', 'update', 'the', 'positions', 'eligibility', 'for', 'lonzo', 'and', 'bledsoe', 'they', 're', 'both', 'listed', 'as', 'point', 'guards', 'and', 'they', 'start', 'alongside', 'each', 'other']]


In [58]:
# Creating Bigram and Trigram model
# bigrams are two words frequently occuring together, trigram is 3 words

# Build the bigram and trigram model 
bigram = gensim.models.Phrases(data_bball_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_bball_words], threshold=100) 

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_bball_words[1]]])

['to', 'say', 'lma', 'has', 'been', 'rough', 'is', 'putting', 'it', 'kindly', 'he', 'seems', 'to', 'have', 'forgotten', 'how', 'to', 'play', 'basketball', 'after', 'top', 'finish', 'last', 'year', 'he', 'is', 'really', 'struggling', 'hard', 'to', 'start', 'the', 'season', 'know', 'it', 'only', 'games', 'but', 'this', 'far', 'his', 'minutes', 'have', 'been', 'low', 'and', 'he', 'not', 'producing', 'in', 'the', 'areas', 'he', 'usually', 'does', 'is', 'it', 'just', 'early', 'season', 'doldrums', 'or', 'is', 'it', 'time', 'to', 'be', 'worried']


In [59]:
# Remove stopwords, make bigrams and lemmatize

# Define function for stopwords, bigrams, trigrams and lemmatization 

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts): 
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts): 
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [60]:
# pipeline 

# Remove Stop words 
data_words_nostops = remove_stopwords(data_bball_words)

# form bigrams 
data_words_bigrams = make_bigrams(data_words_nostops)

data_words_bigrams[1]


['say',
 'lma',
 'rough',
 'putting',
 'kindly',
 'seems',
 'forgotten',
 'play',
 'basketball',
 'top',
 'finish',
 'last',
 'year',
 'really',
 'struggling',
 'hard',
 'start',
 'season',
 'know',
 'games',
 'far',
 'minutes',
 'low',
 'producing',
 'areas',
 'usually',
 'early',
 'season',
 'doldrums',
 'time',
 'worried']

In [61]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm")

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_lemmatized[1]

['say',
 'lma',
 'rough',
 'putting',
 'kindly',
 'seem',
 'forget',
 'play',
 'basketball',
 'top',
 'finish',
 'last',
 'year',
 'really',
 'struggle',
 'hard',
 'start',
 'season',
 'know',
 'game',
 'far',
 'minute',
 'low',
 'produce',
 'area',
 'usually',
 'early',
 'season',
 'doldrum',
 'time',
 'worried']

In [62]:
""" 
Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.
This is used as the input by the LDA Model.
"""

#Create the Dictionary and Corpus needed for Topic Modeling

# Create Dictionary 

dictionary  = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency 
corpus = [dictionary.doc2bow(text) for text in texts]

# View 
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)], [(9, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)], [(19, 7), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 5), (48, 1), (49, 1), (50, 2), (51, 2), (52, 2), (53, 10), (54, 1), (55, 1), (56, 1), (57, 1), (58, 6), (59, 1), (60, 1), (61, 1), (62, 1), (63, 5), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 5), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 10), (76, 1), (77, 1), (78, 1), (79, 1), (80, 5), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1)], [(7, 1), (22, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2)], [(13, 1), (22, 1), (47, 1), (58, 1), (64, 1), (69, 1), (97, 1), (98, 1), (99, 1), (100, 1), 

In [70]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [71]:

NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

In [72]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.030*"team" + 0.027*"game" + 0.020*"tonight" + 0.017*"play"')
(1, '0.122*"trade" + 0.102*"official" + 0.054*"discussion" + 0.049*"january"')
(2, '0.037*"play" + 0.029*"rule" + 0.028*"roster_move" + 0.026*"friday"')
(3, '0.041*"game" + 0.021*"espn" + 0.016*"covid" + 0.015*"player"')
(4, '0.056*"league" + 0.047*"fantasy" + 0.045*"player" + 0.028*"yahoo"')


In [73]:
# 3 Topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.076*"trade" + 0.063*"official" + 0.051*"league" + 0.033*"discussion"')
(1, '0.032*"fantasy" + 0.027*"player" + 0.023*"espn" + 0.021*"yahoo"')
(2, '0.029*"game" + 0.029*"play" + 0.016*"rule" + 0.015*"tonight"')


In [74]:
# 8 Topics the same amount of flair
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 8, id2word=dictionary, passes=15)
ldamodel.save('model8.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.068*"fantasy" + 0.058*"team" + 0.047*"league" + 0.042*"espn"')
(1, '0.035*"league" + 0.027*"yahoo" + 0.022*"fantasy" + 0.022*"game"')
(2, '0.111*"player" + 0.029*"amp" + 0.020*"pick" + 0.019*"good"')
(3, '0.169*"official" + 0.090*"discussion" + 0.081*"january" + 0.061*"february"')
(4, '0.064*"drop" + 0.051*"pd" + 0.022*"cat" + 0.020*"get"')
(5, '0.147*"trade" + 0.035*"help" + 0.028*"league" + 0.022*"good"')
(6, '0.051*"rule" + 0.031*"monday" + 0.024*"friday" + 0.019*"start"')
(7, '0.064*"play" + 0.045*"game" + 0.021*"tonight" + 0.021*"back"')


In [75]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')


In [76]:

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [77]:
lda10 = gensim.models.ldamodel.LdaModel.load('model8.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)